In [48]:
import simpy
import random
import numpy as np
import matplotlib.pyplot as plt
import math

In [49]:
def generate_time_event(tasa_max_llegada, s):
    t = s
    while True:
        u = random.random()
        t = t - (1/tasa_max_llegada)*math.log(u)
        u = random.random()
        if u < t / tasa_max_llegada:
            return t

In [50]:
def average_waiting_time(A, D):
    difference = 0
    for i in range(len(D)):
        difference += D[i][0] - A[i][0]
    return difference / len(D)

In [51]:
from scipy.stats import expon

def simulation_server(tasa_max_llegada, tasa_max_servicio, tiempo):
    tiempo = tiempo
    Na = 0
    Nd = 0
    n = 0
    tasa_max_llegada = tasa_max_llegada
    tasa_max_servicio = tasa_max_servicio
    ta = generate_time_event(tasa_max_llegada, 0)
    td = math.inf
    A = []
    D = []
    tp = 0
    t = 0
    for T in range(tiempo):

        if ta <= td and ta <= T:
            t = ta
            Na = Na + 1
            n += 1
            ta = generate_time_event(tasa_max_llegada, T)
            if n == 1:
                # td = t + random.expovariate(tasa_max_servicio)
                td = t + expon.rvs(scale=1/tasa_max_servicio, size=1)
            A.append([t, Na, Nd, n])
        
        elif td < ta and td <= T:
            t = td
            Nd = Nd + 1
            n -= 1
            if n == 0:
                td = math.inf
            else:
                # td = t + random.expovariate(tasa_max_servicio)
                td = t + expon.rvs(scale=1/tasa_max_servicio, size=1)
            D.append([t, Na, Nd, n])
            
        elif min(ta, td) > T and n > 0:
            t = td
            Nd = Nd + 1
            n -= 1
            if n > 0:
                # td = t + random.expovariate(tasa_max_servicio)
                td = t + expon.rvs(scale=1/tasa_max_servicio, size=1)
            D.append([t, Na, Nd, n])
        
        elif min(ta, td) > T:
            tp = max(t - T, 0)
            
    tiempo_promedio = average_waiting_time(A, D)
    acu = f"Cantidad de solicitudes entrantes: {Na}\nCantidad de solicitudes atendidas: {Nd}\nCantidad de solicitudes en espera: {n}\nTiempo promedio de espera: {tiempo_promedio}"
    print(acu)


In [52]:
simulation_server(100, 40, 60*60)

Cantidad de solicitudes entrantes: 1800
Cantidad de solicitudes atendidas: 1799
Cantidad de solicitudes en espera: 1
Tiempo promedio de espera: [0.02423168]


In [53]:
simulation_server(100, 60, 60*60)

Cantidad de solicitudes entrantes: 1800
Cantidad de solicitudes atendidas: 1799
Cantidad de solicitudes en espera: 1
Tiempo promedio de espera: [0.01616605]


In [54]:
# Parámetros de la simulación
tiempo_simulacion = 3600  # Tiempo de simulación en segundos
tasa_llegada_solicitudes = 40  # Tasa de llegada de solicitudes al sistema (solicitudes por segundo)
tasa_atencion_servidor = 10  # Tasa de atención del servidor (solicitudes por segundo)

# Variables para almacenar métricas
solicitudes_atendidas_por_servidor = 0  # Inicialmente, el servidor no ha atendido ninguna solicitud
tiempo_servidor_ocupado = 0  # Tiempo que estuvo el servidor ocupado
tiempo_servidor_desocupado = 0  # Tiempo que estuvo el servidor desocupado
tiempo_total_solicitudes_en_cola = []  # Tiempo total que las solicitudes estuvieron en cola
total_solicitudes_en_cola = 1

In [63]:
# Proceso para simular la llegada de solicitudes
def llegada_solicitudes(env, tasa_llegada, servidor):
    solicitud_id = 0
    while True:
        arrival = env.now
        yield env.timeout(np.random.exponential(1.0/tasa_llegada))
        solicitud_id += 1
        env.process(atender_solicitud(env, solicitud_id, servidor, arrival))

# Proceso para simular el proceso de atención de una solicitud por el servidor
def atender_solicitud(env, solicitud_id, servidor, arrival):
    llegada_solicitud = env.now
    # print(f"Llegada de solicitud {solicitud_id} en el tiempo {llegada_solicitud}")
    # Calcular tiempo de atención
    tiempo_atencion = np.random.exponential(1.0/tasa_atencion_servidor)

    # Registrar tiempo en que el servidor se ocupó
    global tiempo_servidor_ocupado
    tiempo_servidor_ocupado += tiempo_atencion
    tiempo_total_solicitudes_en_cola.append(llegada_solicitud - arrival)
    # Atender la solicitud
    print(f"arrival:{arrival}, tiempo_atencion:{tiempo_atencion}, tiempo_servidor_ocupado:{tiempo_servidor_ocupado}")
    yield env.timeout(tiempo_atencion)


    # Incrementar el contador de solicitudes atendidas por el servidor
    global solicitudes_atendidas_por_servidor
    solicitudes_atendidas_por_servidor += 1
    # print(f"Atendiendo solicitud {solicitud_id} en el tiempo {env.now}")



In [58]:
# Parámetros de la simulación
tiempo_simulacion = 3600  # Tiempo de simulación en segundos
tasa_llegada_solicitudes = 40  # Tasa de llegada de solicitudes al sistema (solicitudes por segundo)
tasa_atencion_servidor = 100  # Tasa de atención del servidor (solicitudes por segundo)

# Variables para almacenar métricas
solicitudes_atendidas_por_servidor = 0  # Inicialmente, el servidor no ha atendido ninguna solicitud
tiempo_servidor_ocupado = 0  # Tiempo que estuvo el servidor ocupado
tiempo_servidor_desocupado = 0  # Tiempo que estuvo el servidor desocupado
tiempo_total_solicitudes_en_cola = []  # Tiempo total que las solicitudes estuvieron en cola
total_solicitudes_en_cola = 1


# Inicializar la simulación
env = simpy.Environment()

# Crear el servidor
servidor = simpy.Resource(env, capacity=1)

# Iniciar el proceso de llegada de solicitudes
env.process(llegada_solicitudes(env, tasa_llegada_solicitudes, servidor))

# Ejecutar la simulación
env.run(until=tiempo_simulacion)

# Calcular métricas finales
tiempo_total_simulacion = tiempo_simulacion * 1.0
tiempo_total_solicitudes_en_cola = sum(tiempo_total_solicitudes_en_cola)
promedio_tiempo_en_cola = tiempo_total_solicitudes_en_cola / solicitudes_atendidas_por_servidor
promedio_solicitudes_en_cola_por_segundo = total_solicitudes_en_cola / tiempo_total_simulacion
ultimo_momento_salida_solicitud = env.now
tiempo_servidor_desocupado = tiempo_total_simulacion - tiempo_servidor_ocupado


# Mostrar métricas finales
print("\nMétricas finales:")
print(f"a. Solicitudes atendidas por el servidor: {solicitudes_atendidas_por_servidor}")
print(f"b. Tiempo total en que el servidor estuvo ocupado: {tiempo_servidor_ocupado}")
print(f"c. Tiempo total en que el servidor estuvo desocupado: {tiempo_servidor_desocupado}")
print(f"d. Tiempo total que las solicitudes estuvieron en cola: {tiempo_total_solicitudes_en_cola}")
print(f"e. Promedio de tiempo que cada solicitud estuvo en cola: {promedio_tiempo_en_cola}")
print(f"f. Promedio de solicitudes en cola por segundo: {promedio_solicitudes_en_cola_por_segundo}")
print(f"g. Momento de salida de la última solicitud: {ultimo_momento_salida_solicitud}")


Métricas finales:
a. Solicitudes atendidas por el servidor: 143844
b. Tiempo total en que el servidor estuvo ocupado: 1432.6045723318264
c. Tiempo total en que el servidor estuvo desocupado: 2167.3954276681734
d. Tiempo total que las solicitudes estuvieron en cola: 3599.95199820443
e. Promedio de tiempo que cada solicitud estuvo en cola: 0.02502677899811205
f. Promedio de solicitudes en cola por segundo: 0.0002777777777777778
g. Momento de salida de la última solicitud: 3600


In [64]:
# Parámetros de la simulación
tiempo_simulacion = 3  # Tiempo de simulación en segundos
tasa_llegada_solicitudes = 40  # Tasa de llegada de solicitudes al sistema (solicitudes por segundo)
tasa_atencion_servidor = 1  # Tasa de atención del servidor (solicitudes por segundo)

# Variables para almacenar métricas
solicitudes_atendidas_por_servidor = 0  # Inicialmente, el servidor no ha atendido ninguna solicitud
tiempo_servidor_ocupado = 0  # Tiempo que estuvo el servidor ocupado
tiempo_servidor_desocupado = 0  # Tiempo que estuvo el servidor desocupado
tiempo_total_solicitudes_en_cola = []  # Tiempo total que las solicitudes estuvieron en cola
total_solicitudes_en_cola = 1
capacidad = 1

# Inicializar la simulación
env2 = simpy.Environment()

# Crear el servidor
servidor2 = simpy.Resource(env2, capacity=capacidad)

# Iniciar el proceso de llegada de solicitudes
env2.process(llegada_solicitudes(env2, tasa_llegada_solicitudes, servidor2))

# Ejecutar la simulación
env2.run(until=tiempo_simulacion)

# Calcular métricas finales
tiempo_total_simulacion = tiempo_simulacion * 1.0
tiempo_total_solicitudes_en_cola = sum(tiempo_total_solicitudes_en_cola) / capacidad
promedio_tiempo_en_cola = tiempo_total_solicitudes_en_cola / solicitudes_atendidas_por_servidor
promedio_solicitudes_en_cola_por_segundo = total_solicitudes_en_cola / tiempo_total_simulacion
ultimo_momento_salida_solicitud = env2.now
tiempo_servidor_ocupado = tiempo_servidor_ocupado / capacidad
tiempo_servidor_desocupado = tiempo_total_simulacion - tiempo_servidor_ocupado


# Mostrar métricas finales
print("\nMétricas finales:")
print(f"a. Solicitudes atendidas por el servidor: {solicitudes_atendidas_por_servidor}")
print(f"b. Tiempo total en que el servidor estuvo ocupado: {tiempo_servidor_ocupado}")
print(f"c. Tiempo total en que el servidor estuvo desocupado: {tiempo_servidor_desocupado}")
print(f"d. Tiempo total que las solicitudes estuvieron en cola: {tiempo_total_solicitudes_en_cola}")
print(f"e. Promedio de tiempo que cada solicitud estuvo en cola: {promedio_tiempo_en_cola}")
print(f"f. Promedio de solicitudes en cola por segundo: {promedio_solicitudes_en_cola_por_segundo}")
print(f"g. Momento de salida de la última solicitud: {ultimo_momento_salida_solicitud}")

arrival:0, tiempo_atencion:0.12078473581323519
arrival:0.013470479338729625, tiempo_atencion:2.132146622531097
arrival:0.055804141755981265, tiempo_atencion:2.5571425756169646
arrival:0.056751097980402294, tiempo_atencion:1.4527322312902988
arrival:0.0598052052378372, tiempo_atencion:2.0463118070928097
arrival:0.07176212823526851, tiempo_atencion:0.3661174088919374
arrival:0.08577965313670634, tiempo_atencion:1.1158556248551434
arrival:0.13787015468982097, tiempo_atencion:0.19571367656069186
arrival:0.16517959037140323, tiempo_atencion:3.3536820934102938
arrival:0.17402010363847806, tiempo_atencion:0.7162234980245398
arrival:0.1953838131029434, tiempo_atencion:2.7353052759623178
arrival:0.2042536095277795, tiempo_atencion:1.5177024503939678
arrival:0.25691183107110616, tiempo_atencion:0.8661038444836953
arrival:0.28374057837457284, tiempo_atencion:0.7779745651750507
arrival:0.39942788941970325, tiempo_atencion:0.36911785951547815
arrival:0.4325011200676876, tiempo_atencion:0.7210088397